In [ ]:
import tensorflow as tf
import tensorflow.keras

In [ ]:
model = tf.keras.models.load_model('data/model/LSTM2LSTM/model.h5')

In [ ]:
model.summary()

In [ ]:
import os

import pandas as pd

from src.io.path_definition import get_file

In [ ]:
hotel_info = pd.read_csv(get_file(os.path.join('data', 'cancel_dataset_hotel_info.csv')))

cancel_target = pd.read_csv(get_file(os.path.join('data', 'cancel_dataset_target.csv')))
date_feature = pd.read_csv(get_file(os.path.join('data', 'cancel_dataset_date_feature.csv')))
hotel_meta = pd.read_csv(get_file(os.path.join('data', 'cancel_dataset_hotel_info.csv')), index_col=0)

covid_data = pd.read_excel(get_file(os.path.join('data', 'owid-covid-data.xlsx')),
                           #engine='openpyxl'
                           )


def data_preparation(hotel_id: int, date_feature: pd.DataFrame, cancel_target: pd.DataFrame):

    column = f"hotel_{hotel_id}_canceled"

    cancel_target['date'] = cancel_target['date'].apply(lambda x: datetime.strptime(x, '%Y/%m/%d').strftime("%Y/%m/%d"))
    cancel_target.set_index('date', inplace=True)
    hotel_cancel = cancel_target[column].replace("-", np.nan).dropna().astype(int)

    date_feature['date'] = date_feature['date'].apply(lambda x: datetime.strptime(x, '%Y/%m/%d').strftime("%Y/%m/%d"))
    date_feature.set_index('date', inplace=True)
    date_feature = date_feature.loc[hotel_cancel.index]

    date_feature['canceled'] = hotel_cancel   # 原始值

    twn_covid_data = covid_data[covid_data['iso_code']=='TWN']
    twn_covid_data['date'] = twn_covid_data['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime("%Y/%m/%d"))
    twn_covid_data.set_index('date', inplace=True)

    covid_features = []#['new_cases', 'new_deaths']

    date_feature = date_feature.join(twn_covid_data[covid_features].fillna(0))

    num_feature_columns = ['canceled'] + covid_features

    return num_feature_columns, date_feature

In [ ]:
import numpy as np
from datetime import datetime

hotel_id = 57

numerical_features, date_feature = data_preparation(hotel_id, date_feature.copy(), cancel_target.copy())

date_feature = date_feature[numerical_features]

In [ ]:
numerical_features

In [ ]:
model.summary()

In [ ]:
test_size = 28
input_range = 30
lead_time = 0
prediction_time = 7

In [ ]:
import joblib

with open(os.path.join('data', 'model', 'LSTM2LSTM', 'scaler'), mode='rb') as f:
    scaler = joblib.load(f)

In [ ]:
scaler.data_max_

In [ ]:
scaler.data_min_

In [ ]:
scaler.feature_names_in_

In [ ]:
date_feature[scaler.feature_names_in_] = scaler.transform(date_feature[scaler.feature_names_in_])

In [ ]:
date_feature[scaler.feature_names_in_].head(5)

In [ ]:
from train.logic.data_preparation import to_supervised

results = to_supervised(date_feature, input_range=input_range, prediction_time=prediction_time,
                        numerical_features=numerical_features, prediction=False)

In [ ]:
results.keys()

In [ ]:
from train.logic.data_preparation import parse_tf_input

X, y = parse_tf_input(results, prediction=False)

In [ ]:
X['encoder_X_num'].shape

In [ ]:
y.keys()

In [ ]:
y_pred = model.predict(X)

In [ ]:
y_pred.shape

In [ ]:
y['true'].shape

In [ ]:
y_true_extend = np.repeat(y['true'][-test_size:].reshape(-1, 1), len(scaler.scale_), axis=1)
y_true_reshape = scaler.inverse_transform(y_true_extend)[:, 0].reshape(y['true'][-test_size:].shape)

In [ ]:
y_true_reshape.shape

In [ ]:
y_true_reshape[-1]

In [ ]:
y_true_reshape[0]

In [ ]:
y_pred_extend = np.repeat(y_pred[-test_size:].reshape(-1, 1), len(scaler.scale_), axis=1)
y_pred_reshape = scaler.inverse_transform(y_pred_extend)[:, 0].reshape(y_pred[-test_size:].shape)

In [ ]:
y_pred_extend.shape

In [ ]:
y_pred_reshape[0]

In [ ]:
y_pred_reshape.shape

In [ ]:
y_pred_reshape[:, 0, 0]